code for analysing binary veseel SLO image obtained from automorph

In [2]:
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt
from skimage import exposure 
from skimage.morphology import (erosion, dilation, closing, opening)
from PIL import Image
from skimage.draw import circle_perimeter
from skimage import color, img_as_ubyte
import skimage
from skimage import measure
from skimage import io, morphology
from scipy.spatial import distance
from skimage.morphology import skeletonize
from skimage.morphology import skeletonize,remove_small_objects
from skimage import color, img_as_ubyte

In [3]:
from load_images import load_images_from_folder

In [4]:
def show_images(images):
    n: int = len(images)
    f = plt.figure()
    for i in range(n):
        # Debug, plot figure
        f.add_subplot(1, n, i + 1)
        plt.imshow(images[i],cmap='gray', vmin=0, vmax=255)

    plt.show(block=True)

In [5]:
def finding_terminal_points(vessel_skeleton):
    
    count=0 
    terminal_point=[]
    for it_x in range(1, vessel_skeleton.shape[0] - 1):
        for it_y in range(1, vessel_skeleton.shape[1] - 1):
                aux = 0
                if vessel_skeleton[it_x, it_y] == 1:
                    aux += np.sum(vessel_skeleton[it_x - 1:it_x + 1 + 1, it_y - 1:it_y + 1 + 1])
                    if aux == 2:
                       # print('a')
                        point=(it_x,it_y)#itx:row, it_y:column
                        terminal_point.append(point) 
        
       # terminal_point[images_name[i]]=terminal_point
                        
        
        if count==0:
            pass
            
    return terminal_point

In [6]:
def algorithm_A(th_old,im_1,vessel_1,seed,seed_set,  S1,threshold):
    
    condition = False
    th=th_old 
    if th_old>40 :
        condition=True
    if seed[0]+1>=vessel_1.shape[0] or seed[1]+1>=vessel_1.shape[1] or seed[1]-1<=0 or seed[0]-1<=0:
        condition = False
                     
    while condition:
        ###neighboring pixels of seed
        window=im_1[seed[0]-1:seed[0]+2,seed[1]-1:seed[1]+2]
        window1=vessel_1[seed[0]-1:seed[0]+2,seed[1]-1:seed[1]+2]
        ###neighboring pixels not belonging to binary vessel
        p,q=np.where(window==0)        
        if len(p)==0:
                condition=False
                      
        if len(p)!=0:
            ### comparision of the intensity of pixels not belinging to vessel and th
            window2=np.where(window==0,window1,0)
           # print(seed)
            comparison_intensity=abs(np.subtract(window2,th*np.ones((3,3))))
            #print(f'comparison_intensity:{comparison_intensity}')
            m,n=np.where(comparison_intensity<threshold)#10
           # print(f'size of m : {len(m)}')            
            if len(m)==0:
                condition=False               
            if len(m)!=0:
                 
               ### compute new_seed
                mm,nn=np.where(comparison_intensity==comparison_intensity.min().min())
                #print(f'mm:{mm}, nn:{nn}')
                ind_row=mm[0]-1
                ind_col=nn[0]-1
                new_seed=(np.add(seed[0],ind_row),np.add(seed[1],ind_col))
                if new_seed[0]+1>=vessel_1.shape[0] or new_seed[1]+1>=vessel_1.shape[1] or new_seed[1]-1<=0 or new_seed[0]-1<=0:
                    condition = False
                th_new=np.mean([th_old, vessel_1[new_seed[0],new_seed[1]]])
                if len(mm)>1:
                    for ss in range(len(mm)):
                        if ss>=1:
                            ind_row=mm[ss]-1
                            ind_col=nn[ss]-1
                            new_point=(np.add(point[0],ind_row),np.add(point[1],ind_col))
                            S1.append(new_point)
                #print(f'th_new:{th_new}')
                seed=new_seed
                seed_set.append(new_seed)
                th=th_new                
                im_1[new_seed[0],new_seed[1]]=255
                              
                 ### informing S1
                window3=np.uint8(np.multiply(comparison_intensity<threshold,comparison_intensity>comparison_intensity.min().min()))
                pp,qq=np.where(window3==1)#pp:row,qq:col
                for l in range(len(pp)):
                    ind_row=pp[l]-1
                    ind_col=qq[l]-1
                    new_point=(np.add(point[0],ind_row),np.add(point[1],ind_col))
                    S1.append(new_point)
                    
    return S1,seed_set,im_1

In [7]:
###loading data
data_path='./all/'
images,images_name=load_images_from_folder(data_path)

In [8]:
binary_vessel_path='./resize_binary/'
vessel_path='./resize/'
vessel_skeleton_path='./vessel_skeleton/'
path_final_vessel='./final_vessel_result/'
if not os.path.exists(path_final_vessel):
        os.makedirs(path_final_vessel)
path_region_growing='./region_growing_result/'
if not os.path.exists(path_region_growing):
        os.makedirs(path_region_growing)

In [49]:
###analysing the binary vessel image obtained from automorph :
binary_vessel = []
vessel=[]
vessel_skeleton=[]

th=40
threshold=20#15
count=0
for filename in os.listdir(binary_vessel_path):
    #filename= 'bestar_ahang_a_136.OS.png'   
    img_binary=cv2.cvtColor(cv2.imread(os.path.join(binary_vessel_path,filename)), cv2.COLOR_BGR2GRAY)
    img_skeleton=np.uint8(skeletonize(np.where(img_binary>0, 1,0)))
    img_vessel=cv2.cvtColor(cv2.imread(os.path.join(vessel_path,filename)), cv2.COLOR_BGR2GRAY)
    binary_vessel.append(img_binary)
    vessel.append(img_vessel)
    vessel_skeleton.append(img_skeleton)
    ######## computing landmark points
    terminal_point=finding_terminal_points(img_skeleton)
    terminal_point0=np.array(terminal_point)
    #plt.scatter(terminal_point0[:, 1], terminal_point0[:, 0], marker="x", color="red", s=2)
    #plt.imshow(img_skeleton)
    #######region growing algorithm
    im=np.multiply(np.where(img_binary>0,1,0),img_vessel)
    im_1=np.copy(im)
    vessel_1=np.copy(img_vessel)
    seed_set=[]
    for i in range(len(terminal_point)):
        S1=[]
        point=terminal_point[i]
        th_old=img_vessel[point[0],point[1]]
        th=th_old 
        seed = point
        ####Algprithm A
        S1,seed_set,im_1=algorithm_A(th_old,im_1,vessel_1,seed,seed_set,S1,threshold) 
       
        ####Algorithm B
        condition2 = False
        if len(S1) !=0:
            condition2 = True
        
        while condition2:
            S = S1
            S1=[]
            for k in range(len(S)):
                point_s=S[k]
                th1= img_vessel[point_s[0],point_s[1]]
            if th1>40:
                S1,seed_set,im_1=algorithm_A(th1,im_1,vessel_1,point_s,seed_set, S1,threshold)                
            if len(S1) == 0:
                condition2 = False
   
    img = np.uint8(remove_small_objects(im_1>0, 50,connectivity=5)) 
    image_color = color.gray2rgb(img_as_ubyte(np.uint8(img)*255))
    if len(seed_set) != 0:        
        for jj in range(len(seed_set)):            
            image_color = cv2.circle(image_color, (seed_set[jj][1],seed_set[jj][0]), radius=1, color=(0, 0, 255), thickness=-1)
            #plt.scatter(np.array(seed_set)[:, 1], np.array(seed_set)[:, 0], marker="x", color="red", s=0.01) 
       # plt.plot(np.array(seed_set)[:, 1], np.array(seed_set)[:, 0], marker='v', color="white") 
     #plt.imshow(image_color) 
    cv2.imwrite(path_region_growing+filename, image_color)
    cv2.imwrite(path_final_vessel+filename, img*255)
    
    if count==0:
        pass
    